# CNN inference with eGPU - part 2

Today we'll run neural networks inference on NVIDIA Jetson Nano eGPUs.

You can find out more here: https://www.nvidia.com/pl-pl/autonomous-machines/embedded-systems/jetson-nano-developer-kit/

First, install and import nessessery libraries.

In [ ]:
!pip3 install --upgrade tensorrt
!pip install tensorflow-datasets
import tensorflow as tf
import tensorrt

Before we start, chech if the GPU is available (we going to need it!).

In [ ]:
print(tf.config.list_physical_devices('GPU'))

Today, we are going to try a bit bigger model and harder dataset. We'll try running Imagenet classification with MobileNet network.

More about the dataset: https://www.image-net.org/

More about the network: https://keras.io/api/applications/mobilenet/

It would take some time to train this model, so we'll just use ready, pretrained neural network. Use `tf.keras.applications.MobileNet()` function to create `CNN` instance (just study the link above). Use `include_top=True`, `weights="imagenet"` and `classes=1000`. Based on documentation answer the question - what is model's input size and what is it's output size?



In [ ]:
CNN = tf.keras.applications.MobileNet( ... )

Now, let's download the dataset with `tfds` module. Use `tfds.load()` function with `imagenet_v2` dataset name (this is the dataset used for MobileNet training), `split='test[70%:]` (we need just 3000 samples), and `shuffle_files=True` and `as_supervised=True` parameters.

In [ ]:
import tensorflow_datasets as tfds
ds = tfds.load( ... )

Perfect. Now, we have both pretrained model and test dataset ready. We can benchmark the model. Implement benchmarking similarly as in previous lab, but:
- calulate not only throughput, and TOP1 accuracy, but also TOP5 accuracy (is correct label found in 5 classes with highest prediction probability?). The `((-preds[0]).argsort()[:5])` function may prove useful here.
- you can loop through dataset with `for image, label in tfds.as_numpy(ds):`
- each image should be resized to model input size and then reshaped to `(1,input_size, input_size, nr_of_channels)` before `predict` function. Just before model's input use `preprocess_input()`

In [ ]:
from tensorflow.keras.applications.mobilenet import preprocess_input, decode_predictions

# Benchmarking throughput and TOP1/TOP5 accuracy
import time
import numpy as np
import cv2

N_warmup_run = 50
N_run = 500

for image, label in tfds.as_numpy(ds):
  # TO DO


print( ... ) # Throughput
print( ... ) # TOP1 accuracy
print( ... ) # TOP1 accuracy

After benchmarking - save model with `CNN.save()`. We got familiar with MobileNet and ImageNet. Now we can carry on with Jetson Nano. Show this part of exercise to the teacher and ask for Jeston Nano board.

In [ ]:
SAVED_MODEL_DIR="saved_model"
CNN.save(SAVED_MODEL_DIR)

Our task is to run inference on Jetson Nano with TensorRT model. You are going to:
1. Prepare the Jetson Nano board.
2. Convert Mobilenet with TensorRT.
3. Run inference on example image.
4. **Extended** Benchmark Jetson Nano inference.
5. **Extension exercise** Connect camera and run live classification.

1. Prepare the Jetson Nano board
- First, take Jetson Nano board, connect it to power source, internet, monitor, mouse and keyboard.
- Log in to Jetson and finish OS instalation (the boards were not used yet).
- Open terminal and add cuda to PATH `export PATH=$PATH:/usr/local/cuda-10/bin`. Verify CUDA with `nvcc --version`.
- Download NVIDIA Tensorflow docker `sudo docker pull nvcr.io/nvidia/l4t-tensorflow:r32.7.1-tf2.7-py3`
- Run NVIDIA Tensorflow docker with `sudo docker run -it --rm --runtime nvidia --network host -v /home/nano/Documents:/home/ nvcr.io/nvidia/l4t-tensorflow:r32.7.1-tf2.7-py3`. Now, you can use TensorRT, TensorFlow and CUDA in this terminal window. Moreover, the path `/home/nano/Documents/` outside of docker is linked to `/home/` inside of docker.

2. Convert MobileNet with TensorRT

Use the code from Lab6 to convert `MobileNet` model to TensorRT. Start with FP32 model (to make it simple). You can move `save_model` created here to Jetson or create CNN on Jetson (`tf.keras.applications.MobileNet()`)

3. Run inference on example image.
- Warmup model with dummy imput (`np.random`)
- Pass image to `infer` and analize network output.
- You can get example images from imagenet from `https://github.com/EliSchwartz/imagenet-sample-images.git` repository. Use PIL to read and resize images.

4. **Extension exercise** - Loop thorugh all images, benchmark TensorRT model.

5. **Extension exercise** - Implement simple application that uses USB camera and TensorRT for live classification.